In [1]:
import os
import numpy as np
import pandas as pd
# Pandas Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize

import json
import requests
import re
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline 

import folium
from folium import plugins
from folium.plugins import HeatMap

from sklearn.cluster import KMeans

print('Libraries Imported.')

Libraries Imported.


In [2]:
# Define Url 
URL = r'https://developer.foursquare.com/docs/build-with-foursquare/categories/'

In [21]:
page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <style data-href="/docs/styles.7a34e2faa5fd2d9bc3fe.css" id="gatsby-global-css">
   @font-face{font-family:AUTHENTICSans-60;font-style:normal;font-weight:300;src:url(data:font/woff2;base64,d09GMk9UVE8AACbwAAkAAAAAR5gAACapABM52wAAAAAAAAAAAAAAAAAAAAAAAAAADfF9BmAAjggBNgIkA4UYBAYFhngHIBv0RkUjddg4AL4ZtjKikpMkirLBmJj9fzjgxhCtgf4VGtsdogUhThCdcUNJClOGzliDidCOU9iZaMQq+iZTQhT/MrOxJuyJV1TyklAjGjg2FzgN7fPfUOLb6d4JHeJ+e0hSNAdwm5uEZE/ZSJMWrMDCiv6bJ/e8w3p3zcvDf56m65ufP5OuzUKGWUmWGdTWECtQxNoGirh0BQIrljW6QvGiBS9VS5tJUqfU0CLaBi2cN9wJe6VbyPQU7b2VMa8giB68yhE2phuk7p4n/tf43Zm3WDaRzE9wOk0Tnj2bVdFafylYXogAL5sNCuAqPZUSL1EJgP4H+BHIw9uu3AgvfxozXTBBuJvITKhZrHaDKpyUMpeDwn1o7Vd9g3mcCG0hUQKh0Qr7sZ2P7iIq6Q9kr0TxaiHuLvjZzvl+3NtdtHihce1CKqL//1q/OoOo37CHAQ9tIYmVSEjMW/m8mfMGZr/OfnHbnV3/4ohaFPekiZCkqTciJVEiNCIlasgJcJmKM

In [39]:
def test(s):
    ul = s.find('ul',  {'class': 'VenueCategories__Wrapper-sc-1ysxg0y-0 dikXMT'})
    li = ul.findAll('li', recursive=False)
    if not li: return None
    rtn = []
    for item in li: 
        item_text = item.find('h3').text
        o = test(item)
        if o is None:
            rtn.append(item_text)
        else:
            rtn.append({item_text: o})
    return rtn
        
res = test(soup)
res

[{'Arts & Entertainment': ['Amphitheater',
   'Aquarium',
   'Arcade',
   'Art Gallery',
   'Bowling Alley',
   'Casino',
   'Circus',
   'Comedy Club',
   'Concert Hall',
   'Country Dance Club',
   'Disc Golf',
   'Exhibit',
   'General Entertainment',
   'Go Kart Track',
   'Historic Site',
   'Karaoke Box',
   'Laser Tag',
   'Memorial Site',
   'Mini Golf',
   {'Movie Theater': ['Drive-in Theater', 'Indie Movie Theater', 'Multiplex']},
   {'Museum': ['Art Museum',
     'Erotic Museum',
     'History Museum',
     'Planetarium',
     'Science Museum']},
   {'Music Venue': ['Jazz Club', 'Piano Bar', 'Rock Club']},
   'Pachinko Parlor',
   {'Performing Arts Venue': ['Dance Studio',
     'Indie Theater',
     'Opera House',
     'Theater']},
   'Pool Hall',
   {'Public Art': ['Outdoor Sculpture', 'Street Art']},
   'Racecourse',
   'Racetrack',
   'Roller Rink',
   'Salsa Club',
   'Samba School',
   {'Stadium': ['Baseball Stadium',
     'Basketball Stadium',
     'Cricket Ground',
  

In [46]:
page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')

ul = soup.find('ul',  {'class': 'VenueCategories__Wrapper-sc-1ysxg0y-0 dikXMT'})
general_categories = ul.findAll('li', recursive=False)
for gen in general_categories:
    print(gen.find('h3').text)
    # Get Sub categories
    sub_list = gen.find('ul',  {'class': 'VenueCategories__Wrapper-sc-1ysxg0y-0 dikXMT'})
    sub_categories = sub_list.findAll('li', recursive=True)
    for sub in sub_categories:
        print('-', sub.find('h3').text)

Arts & Entertainment
- Amphitheater
- Aquarium
- Arcade
- Art Gallery
- Bowling Alley
- Casino
- Circus
- Comedy Club
- Concert Hall
- Country Dance Club
- Disc Golf
- Exhibit
- General Entertainment
- Go Kart Track
- Historic Site
- Karaoke Box
- Laser Tag
- Memorial Site
- Mini Golf
- Movie Theater
- Drive-in Theater
- Indie Movie Theater
- Multiplex
- Museum
- Art Museum
- Erotic Museum
- History Museum
- Planetarium
- Science Museum
- Music Venue
- Jazz Club
- Piano Bar
- Rock Club
- Pachinko Parlor
- Performing Arts Venue
- Dance Studio
- Indie Theater
- Opera House
- Theater
- Pool Hall
- Public Art
- Outdoor Sculpture
- Street Art
- Racecourse
- Racetrack
- Roller Rink
- Salsa Club
- Samba School
- Stadium
- Baseball Stadium
- Basketball Stadium
- Cricket Ground
- Football Stadium
- Hockey Arena
- Rugby Stadium
- Soccer Stadium
- Tennis Stadium
- Track Stadium
- Theme Park
- Theme Park Ride / Attraction
- Tour Provider
- Water Park
- Zoo
- Zoo Exhibit
College & University
- Coll

In [49]:
page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')

def get_sub_categories(foobar):
    sub_list = foobar.find('ul',  {'class': 'VenueCategories__Wrapper-sc-1ysxg0y-0 dikXMT'})
    sub_categories = [sub.find('h3').text for sub in sub_list.findAll('li', recursive=True)]
    return sub_categories

ul = soup.find('ul',  {'class': 'VenueCategories__Wrapper-sc-1ysxg0y-0 dikXMT'})
general_categories = ul.findAll('li', recursive=False)
general_venue_categories = {gen.find('h3').text: get_sub_categories(gen) for gen in general_categories}

general_venue_categories



{'Arts & Entertainment': ['Amphitheater',
  'Aquarium',
  'Arcade',
  'Art Gallery',
  'Bowling Alley',
  'Casino',
  'Circus',
  'Comedy Club',
  'Concert Hall',
  'Country Dance Club',
  'Disc Golf',
  'Exhibit',
  'General Entertainment',
  'Go Kart Track',
  'Historic Site',
  'Karaoke Box',
  'Laser Tag',
  'Memorial Site',
  'Mini Golf',
  'Movie Theater',
  'Drive-in Theater',
  'Indie Movie Theater',
  'Multiplex',
  'Museum',
  'Art Museum',
  'Erotic Museum',
  'History Museum',
  'Planetarium',
  'Science Museum',
  'Music Venue',
  'Jazz Club',
  'Piano Bar',
  'Rock Club',
  'Pachinko Parlor',
  'Performing Arts Venue',
  'Dance Studio',
  'Indie Theater',
  'Opera House',
  'Theater',
  'Pool Hall',
  'Public Art',
  'Outdoor Sculpture',
  'Street Art',
  'Racecourse',
  'Racetrack',
  'Roller Rink',
  'Salsa Club',
  'Samba School',
  'Stadium',
  'Baseball Stadium',
  'Basketball Stadium',
  'Cricket Ground',
  'Football Stadium',
  'Hockey Arena',
  'Rugby Stadium',
  